In [1]:
import pandas as pd
# import scispacy
import spacy
import re
import nltk
from spacy import displacy
from spacy.matcher import Matcher
from nltk.tokenize import sent_tokenize, word_tokenize

In [2]:
med_transcript = pd.read_csv("mtsamples.csv", index_col=0)

In [3]:
med_transcript.info()
med_transcript.head(1)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4999 entries, 0 to 4998
Data columns (total 5 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   description        4999 non-null   object
 1   medical_specialty  4999 non-null   object
 2   sample_name        4999 non-null   object
 3   transcription      4966 non-null   object
 4   keywords           3931 non-null   object
dtypes: object(5)
memory usage: 234.3+ KB


,description,medical_specialty,sample_name,transcription,keywords
0,A 23-year-old white female presents with comp...,Allergy / Immunology,Allergic Rhinitis,"SUBJECTIVE:, This 23-year-old white female pr...","allergy / immunology, allergic rhinitis, aller..."


In [4]:
# Drug Detection Model
nlp = spacy.load("en_ner_bc5cdr_md")


c:\Users\RAZAK\Desktop\venv\venv\lib\site-packages\spacy\util.py:833: UserWarning: [W095] Model 'en_ner_bc5cdr_md' (0.4.0) was trained with spaCy v3.0 and may not be 100% compatible with the current version (3.2.1). If you see errors or degraded performance, download a newer compatible model or retrain your custom model with the current spaCy version. For more details and available updates, run: python -m spacy validate
  warnings.warn(warn_msg)


In [5]:
pattern = [{'ENT_TYPE':'CHEMICAL'}, {'LIKE_NUM': False} ,{'IS_ASCII': True}]
matcher = Matcher(nlp.vocab)
matcher.add("DRUG_DOSE", [pattern])

In [6]:
def pdftoword(pdf, pg):
    from io import StringIO
    from pdfminer.converter import TextConverter
    from pdfminer.layout import LAParams
    from pdfminer.pdfdocument import PDFDocument
    from pdfminer.pdfinterp import PDFResourceManager, PDFPageInterpreter
    from pdfminer.pdfpage import PDFPage
    from pdfminer.pdfparser import PDFParser

    output_string = StringIO()
    with open(pdf, 'rb') as in_file:
        parser = PDFParser(in_file)
        doc = PDFDocument(parser)
        rsrcmgr = PDFResourceManager()
        device = TextConverter(rsrcmgr, output_string, laparams=LAParams())
        interpreter = PDFPageInterpreter(rsrcmgr, device)
        for i,page in enumerate(PDFPage.create_pages(doc)):
            if i == pg:
                interpreter.process_page(page)
    return output_string.getvalue()

lists = word_tokenize(pdftoword('r.pdf', 0))

new_list_nodates = filter(
    lambda x: not re.match('[0-9]{2}[\/,:][0-9]{2}[\/,:][0-9]{2,4}', x),
    lists)
# print(list)

text = ""
for word in new_list_nodates :
    text += word + ' '
text

'RAZAK Speciality clinic # 11 , guppe road , rvce campus bangalore Doctor name : Razak Date : 22-02-2022 Patient name : Shivnath Age : 69 yrs prescription : Marijuana - 4 times Crocin - twice a day Dolo 650 - 100 times a day Volini – when required price : 1000/- '

In [7]:
nlp_ob = nlp(text)
displacy.render(nlp_ob[:], style='ent', jupyter=True)

In [8]:
pattern = [{'ENT_TYPE':'CHEMICAL'}, {'IS_ASCII': True}]
matcher = Matcher(nlp.vocab)
matcher.add("DRUG_DOSE", [pattern])

In [93]:
'''Drug Name &  Dose Extraction'''
matches = matcher(nlp_ob)
for name in nlp_ob.ents:
    if name.label_ == 'CHEMICAL':
        print(name)

for match_id, start, end in matches:
    string_id = nlp.vocab.strings[match_id]  # get string representation
    span = nlp_ob[start:end]  # the matched span
    print(string_id, start, end, span.text) 

Crocin
DRUG_DOSE 37 39 Crocin -


In [103]:
# Person recognition   
nlp5 = spacy.load('en_core_web_sm')

In [104]:
''' Doctors Name Extraction '''
def find_dr(word):
    for i1, w in enumerate(word):
        if (word[i1:i1+3]).lower() == 'dr.' :
            return i1
    return -1

names_doc_tokens = word_tokenize(pdftoword('Sample Patient Profiles and Prescriptions.pdf', 5))
names_doc_text = ""

for token_idx,w in enumerate(names_doc_tokens):
    if (find_dr(w) == 0) and (len(w) <= 15) :
        break

for word in names_doc_tokens :
    names_doc_text += word + ' '

doctor = names_doc_tokens[token_idx]
# print(doctor)
nlp_names = nlp5(names_doc_tokens[token_idx] + ' ' + names_doc_tokens[token_idx + 1] + '  ' + names_doc_tokens[token_idx + 2])
# print(nlp_names)
displacy.render(nlp_names[:], style='ent', jupyter=True)
for name in nlp_names.ents:
    if name.label_ == 'PERSON':
        doctor = doctor + ' ' + str(name)
print(doctor)


Dr. Best


In [105]:
import dateutil.parser
'''Extract Date'''

def find_date(word):
    for i1, w in enumerate(word):
        if (word[i1:i1+4]).lower() == 'date' :
            # print(word)
            return i1
    return -1

dates_tokens = word_tokenize(pdftoword('Sample Patient Profiles and Prescriptions.pdf', 5))
dates_text = ""


for token_idx,w in enumerate(dates_tokens):
    if (find_date(w) != -1):
        break
probable_dates = [dates_tokens[token_idx + 1] , dates_tokens[token_idx + 2]]
# print(dates_tokens)
date = ''
for probable_date in probable_dates:
    try:
        date = dateutil.parser.parse(probable_date).date()
        break
    except:
        pass
print(date)

2013-08-13


In [15]:
import csv

In [17]:
'''Starting Training using uci drug dataset'''

df_uci = pd.read_csv("drugsComTrain_raw.tsv", sep='\t')
df_uci.head()

,Unnamed: 0,drugName,condition,review,rating,date,usefulCount
0,206461,Valsartan,Left Ventricular Dysfunction,"""It has no side effect, I take it in combinati...",9.0,"May 20, 2012",27
1,95260,Guanfacine,ADHD,"""My son is halfway through his fourth week of ...",8.0,"April 27, 2010",192
2,92703,Lybrel,Birth Control,"""I used to take another oral contraceptive, wh...",5.0,"December 14, 2009",17
3,138000,Ortho Evra,Birth Control,"""This is my first time using any form of birth...",8.0,"November 3, 2015",10
4,35696,Buprenorphine / naloxone,Opiate Dependence,"""Suboxone has completely turned my life around...",9.0,"November 27, 2016",37


In [ ]:
nlp.pipe_names

['tok2vec', 'tagger', 'attribute_ruler', 'lemmatizer', 'parser', 'ner']

In [ ]:
ner = nlp.get_pipe('ner')


In [18]:
ex = "Nexplanon 650mg Crocin"
displacy.render(nlp(ex)[:], style='ent', jupyter=True)

In [19]:
def process_review(review):
    processed_token = []
    for token in review.split():
        token = ''.join(e.lower() for e in token if e.isalnum())
        processed_token.append(token)
    return ' '.join(processed_token)

In [20]:
process_review("Nexplanon 650mg Crocin")

'nexplanon 650mg crocin'

In [21]:
all_drugs_uci = df_uci['drugName'].unique().tolist()
all_drugs_uci = [x.lower() for x in all_drugs_uci]
all_drugs_uci

['valsartan',
 'guanfacine',
 'lybrel',
 'ortho evra',
 'buprenorphine / naloxone',
 'cialis',
 'levonorgestrel',
 'aripiprazole',
 'keppra',
 'ethinyl estradiol / levonorgestrel',
 'topiramate',
 'l-methylfolate',
 'pentasa',
 'dextromethorphan',
 'nexplanon',
 'liraglutide',
 'trimethoprim',
 'amitriptyline',
 'lamotrigine',
 'nilotinib',
 'atripla',
 'trazodone',
 'etonogestrel',
 'etanercept',
 'tioconazole',
 'azithromycin',
 'eflornithine',
 'daytrana',
 'ativan',
 'imitrex',
 'sertraline',
 'toradol',
 'viberzi',
 'mobic',
 'dulcolax',
 'morphine',
 'moviprep',
 'trilafon',
 'fluconazole',
 'contrave',
 'clonazepam',
 'metaxalone',
 'venlafaxine',
 'ledipasvir / sofosbuvir',
 'symbyax',
 'tamsulosin',
 'doxycycline',
 'dulaglutide',
 'intuniv',
 'buprenorphine',
 'qvar',
 'opdivo',
 'pyridium',
 'latuda',
 'bupropion',
 'implanon',
 'effexor xr',
 'drospirenone / ethinyl estradiol',
 'nuvaring',
 'prepopik',
 'tretinoin',
 'gildess fe 1 / 20',
 'ethinyl estradiol / norgestimate'

In [22]:
df_uci['review']

0         "It has no side effect, I take it in combinati...
1         "My son is halfway through his fourth week of ...
2         "I used to take another oral contraceptive, wh...
3         "This is my first time using any form of birth...
4         "Suboxone has completely turned my life around...
                                ...                        
161292    "I wrote my first report in Mid-October of 201...
161293    "I was given this in IV before surgey. I immed...
161294    "Limited improvement after 4 months, developed...
161295    "I&#039;ve been on thyroid medication 49 years...
161296    "I&#039;ve had chronic constipation all my adu...
Name: review, Length: 161297, dtype: object

In [106]:
count = 0

'''
TRAIN_DATA = [
    ('Who is Nishanth?', {'entities': [(7, 15, 'PERSON')]})
]
'''

TRAIN_DATA = []

for _, item in df_uci.iterrows():
    ent_dict = {}
    if count < 5000:
        # print(_, item['drugName'])
        review = process_review(item['review'])
        #Locate drugs and their positions once and add to the visited items.
        visited_items = []
        #In case there is more than 1 entity 
        entities = [] 
        for token in review.split():
            if token in all_drugs_uci:
                for i in re.finditer(token, review):
                    if token not in visited_items:
                        # print(i[0], i.span()[0], i.span()[1])
                        entity = (i.span()[0], i.span()[1], 'CHEMICAL')
                        visited_items.append(token)
                        entities.append(entity)
        if len(entities) > 0:
            ent_dict['entities'] = entities
            train_item = (review, ent_dict)
            TRAIN_DATA.append(train_item)
    else :
        break
    count+=1

In [24]:
TRAIN_DATA

[('it has no side effect i take it in combination of bystolic 5 mg and fish oil',
  {'entities': [(50, 58, 'CHEMICAL')]}),
 ('my son is halfway through his fourth week of intuniv we became concerned when he began this last week when he started taking the highest dose he will be on for two days he could hardly get out of bed was very cranky and slept for nearly 8 hours on a drive home from school vacation very unusual for him i called his doctor on monday morning and she said to stick it out a few days see how he did at school and with getting up in the morning the last two days have been problem free he is much more agreeable than ever he is less emotional a good thing less cranky he is remembering all the things he should overall his behavior is better we have tried many different medications and so far this is the most effective',
  {'entities': [(45, 52, 'CHEMICAL')]}),
 ('i used to take another oral contraceptive which had 21 pill cycle and was very happy very light periods max 5 d

In [30]:
import random
from spacy.training import Example

In [107]:
def train_spacy(data, iterations):
    TRAIN_DATA = data
    nlp = spacy.blank("en")
    if "ner" not in nlp.pipe_names:
        # ner = nlp.create_pipe('ner')
        # nlp.add_pipe(ner, last=True)
        ner = nlp.add_pipe("ner")
    for _, annotations in TRAIN_DATA:
        for ent in annotations.get('entities'):
            ner.add_label(ent[2])
    
    optimizer = nlp.initialize()
    for itn in range(iterations):
        random.shuffle(TRAIN_DATA)
        losses = {}
        for raw_text, entity_offsets in TRAIN_DATA:
            doc = nlp.make_doc(raw_text)
            example = Example.from_dict(doc, entity_offsets)
            nlp.update([example], sgd=optimizer, losses = losses)
            print(losses)
    return nlp



In [108]:
nlp2 = train_spacy(TRAIN_DATA, 1)

{'ner': 65.833336353302}
{'ner': 156.8611137866974}
{'ner': 250.11032629013062}
{'ner': 348.0228058695793}
{'ner': 427.4006686806679}
{'ner': 488.2709004878998}
{'ner': 521.37824010849}
{'ner': 532.0608391314745}
{'ner': 539.353452173993}
{'ner': 543.7164680428104}
{'ner': 545.7498289012165}
{'ner': 547.7447399126783}
{'ner': 551.7403046215594}
{'ner': 553.7391234545955}
{'ner': 557.7381499448018}
{'ner': 563.7367717419169}
{'ner': 567.732831317329}
{'ner': 569.7317943033331}
{'ner': 573.7174392526349}
{'ner': 575.7089227248947}
{'ner': 577.6748248277871}
{'ner': 579.6443841892068}
{'ner': 581.5563786888414}
{'ner': 585.3387395713543}
{'ner': 598.5206309025415}
{'ner': 604.2292499600625}
{'ner': 608.0257816928969}
{'ner': 611.8556555386639}
{'ner': 619.355195359636}
{'ner': 622.980854021447}
{'ner': 626.608467175265}
{'ner': 628.5645987399874}
{'ner': 630.5405806995512}
{'ner': 632.5201794624821}
{'ner': 636.2689324552829}
{'ner': 640.2175961521167}
{'ner': 644.0596002351946}
{'ner': 6

c:\Users\RAZAK\Desktop\venv\venv\lib\site-packages\spacy\training\iob_utils.py:141: UserWarning: [W030] Some entities could not be aligned in the text "after my wellness checkup and blood analysis i was..." with entities "[(85, 87, 'CHEMICAL')]". Use `spacy.training.offsets_to_biluo_tags(nlp.make_doc(text), entities)` to check the alignment. Misaligned entities ('-') will be ignored during training.
  warnings.warn(


{'ner': 863.6677159882504}
{'ner': 866.1765509771975}
{'ner': 868.6237764457985}
{'ner': 868.7480734154274}
{'ner': 868.7545862076715}
{'ner': 869.0703709756791}
{'ner': 870.4346377006726}
{'ner': 872.4243871120774}
{'ner': 874.894280440698}
{'ner': 874.9112276609715}
{'ner': 880.9828176645542}
{'ner': 886.2904845077747}
{'ner': 895.811836594763}
{'ner': 898.1105428834899}
{'ner': 898.1204116628232}
{'ner': 902.4024487538659}
{'ner': 904.2388746307195}
{'ner': 906.1494326779442}
{'ner': 908.132343220251}
{'ner': 910.071946997847}
{'ner': 912.0711332541307}
{'ner': 912.0748679321176}
{'ner': 914.8033751130166}
{'ner': 916.9910229440605}
{'ner': 918.8126783061729}
{'ner': 920.6138073315076}
{'ner': 922.6033381348191}
{'ner': 928.4034018523334}
{'ner': 931.248144595208}
{'ner': 931.3422706688195}
{'ner': 931.4026691569813}
{'ner': 935.075559827379}
{'ner': 940.5146422890637}
{'ner': 940.6361781340349}
{'ner': 942.9638983292423}
{'ner': 944.9066159520124}
{'ner': 946.4260836875453}
{'ner':

c:\Users\RAZAK\Desktop\venv\venv\lib\site-packages\spacy\training\iob_utils.py:141: UserWarning: [W030] Some entities could not be aligned in the text "i started gilotrifafatinib 7 days ago i can feel t..." with entities "[(10, 18, 'CHEMICAL'), (279, 289, 'CHEMICAL'), (62...". Use `spacy.training.offsets_to_biluo_tags(nlp.make_doc(text), entities)` to check the alignment. Misaligned entities ('-') will be ignored during training.
  warnings.warn(


{'ner': 1051.4010871161229}
{'ner': 1055.3247202168232}
{'ner': 1057.5268937998885}
{'ner': 1057.9290008829194}
{'ner': 1058.4050398258255}
{'ner': 1063.452017126853}
{'ner': 1064.7539225550897}
{'ner': 1066.753489831805}
{'ner': 1071.6544438202227}
{'ner': 1074.3287547930047}
{'ner': 1080.4126199590585}
{'ner': 1080.8344636277334}
{'ner': 1085.2769032261267}
{'ner': 1085.2870288300817}
{'ner': 1086.0499237417696}
{'ner': 1086.17446916697}
{'ner': 1086.8363977969707}
{'ner': 1088.3360186290347}
{'ner': 1089.2972858819076}
{'ner': 1090.2222499645616}
{'ner': 1092.2242362875782}
{'ner': 1093.4395754421635}
{'ner': 1095.680729706437}
{'ner': 1097.1831525544221}
{'ner': 1101.0643493309512}
{'ner': 1101.3226668245027}
{'ner': 1101.448979651841}
{'ner': 1101.6077221197515}
{'ner': 1102.364983214165}
{'ner': 1102.5061264103742}
{'ner': 1103.5354749282903}
{'ner': 1106.0597194455543}
{'ner': 1109.3683248096645}
{'ner': 1111.3742824797907}
{'ner': 1111.651679243361}
{'ner': 1114.4379921102436}


...

{'ner': 3631.7771724760983}
{'ner': 3633.157103151285}
{'ner': 3635.016599029509}
{'ner': 3636.802432222364}
{'ner': 3638.651846333244}
{'ner': 3640.6516383237017}
{'ner': 3642.832054631939}
{'ner': 3643.8084029545985}
{'ner': 3644.0996959402123}
{'ner': 3644.1151642877176}
{'ner': 3646.613497270911}
{'ner': 3647.4304213765286}
{'ner': 3647.959386763075}
{'ner': 3647.987630149424}
{'ner': 3647.991549471877}
{'ner': 3648.8798931398837}
{'ner': 3649.5970723625933}
{'ner': 3651.329219839647}
{'ner': 3651.807505487052}
{'ner': 3665.78141594672}
{'ner': 3666.3090811428247}
{'ner': 3668.5434281624284}
{'ner': 3668.564581836616}
{'ner': 3669.82604880713}
{'ner': 3669.8777702657026}
{'ner': 3669.878050051921}
{'ner': 3676.292051279114}
{'ner': 3676.7918910184308}
{'ner': 3676.793169089247}
{'ner': 3678.7315189170968}
{'ner': 3678.805462659713}
{'ner': 3678.896772755113}
{'ner': 3680.6453894167134}
{'ner': 3682.9891023363502}
{'ner': 3688.8074488238326}
{'ner': 3688.8373657214484}
{'ner': 3689.

c:\Users\RAZAK\Desktop\venv\venv\lib\site-packages\spacy\training\iob_utils.py:141: UserWarning: [W030] Some entities could not be aligned in the text "valiumis a life saver instant calming from anxiety..." with entities "[(0, 6, 'CHEMICAL')]". Use `spacy.training.offsets_to_biluo_tags(nlp.make_doc(text), entities)` to check the alignment. Misaligned entities ('-') will be ignored during training.
  warnings.warn(


{'ner': 3768.144069628006}
{'ner': 3768.533798867345}
{'ner': 3770.351202932917}
{'ner': 3773.707187422796}
{'ner': 3775.4836655725917}
{'ner': 3775.504757032797}
{'ner': 3775.882379365787}
{'ner': 3776.0108530003413}
{'ner': 3776.014793210492}
{'ner': 3776.6671019034625}
{'ner': 3776.6683007130882}
{'ner': 3776.668657727343}
{'ner': 3776.9254858470604}
{'ner': 3776.9482845896046}
{'ner': 3781.1453393669717}
{'ner': 3783.0215893853974}
{'ner': 3783.9924294545535}
{'ner': 3785.991885035944}
{'ner': 3790.861852430718}
{'ner': 3791.3320370725796}
{'ner': 3793.331893407744}
{'ner': 3793.460903979571}
{'ner': 3795.460841022113}
{'ner': 3795.92881148997}
{'ner': 3796.9666107881267}
{'ner': 3796.984921353258}
{'ner': 3797.2467115453665}
{'ner': 3797.259641792726}
{'ner': 3799.2638876149927}
{'ner': 3799.2638881162648}
{'ner': 3804.424898158101}
{'ner': 3804.427113215265}
{'ner': 3804.5344817981545}
{'ner': 3804.6920356332257}
{'ner': 3804.692531657286}
{'ner': 3805.874467324618}
{'ner': 3805.

c:\Users\RAZAK\Desktop\venv\venv\lib\site-packages\spacy\training\iob_utils.py:141: UserWarning: [W030] Some entities could not be aligned in the text "i was prescribed lyrica in conjunction with nortri..." with entities "[(17, 23, 'CHEMICAL'), (44, 57, 'CHEMICAL'), (113,...". Use `spacy.training.offsets_to_biluo_tags(nlp.make_doc(text), entities)` to check the alignment. Misaligned entities ('-') will be ignored during training.
  warnings.warn(


{'ner': 3831.690829769783}
{'ner': 3833.4855159215253}
{'ner': 3840.1840406115375}
{'ner': 3844.1787197891927}
{'ner': 3845.0597986759653}
{'ner': 3845.060004661532}
{'ner': 3845.061745125055}
{'ner': 3848.995176569772}
{'ner': 3850.3549170236533}
{'ner': 3850.411959800424}
{'ner': 3850.641390377493}
{'ner': 3852.615881955321}
{'ner': 3854.625221421013}
{'ner': 3855.248814832894}
{'ner': 3858.544582765718}
{'ner': 3862.1154722861256}
{'ner': 3863.0966570864434}
{'ner': 3867.818278378715}
{'ner': 3870.1716427211973}
{'ner': 3871.907793745744}
{'ner': 3872.0107575283255}
{'ner': 3873.8378332247794}
{'ner': 3874.899089379504}
{'ner': 3875.0053118469123}
{'ner': 3876.286615734497}
{'ner': 3876.5150734215085}
{'ner': 3878.1779905733115}
{'ner': 3880.1373519206572}
{'ner': 3881.951965831496}
{'ner': 3885.186679009083}
{'ner': 3885.186778196834}
{'ner': 3886.903140486595}
{'ner': 3887.1064767383496}
{'ner': 3887.2278930382936}
{'ner': 3888.330393497556}
{'ner': 3891.3256553956544}
{'ner': 389

In [111]:
nlp2.to_disk("trained_model_5000")

In [99]:
c = spacy.load("trained_model_1000")

In [112]:
displacy.render(nlp2(ex)[:], style='ent', jupyter=True)

In [62]:
import numpy as np
 
# function to get unique values
def unique(list1):
    x = np.array(list1)
    return np.unique(x)

In [91]:
alldrugs = []
alldrugs = set(alldrugs)
nlpo = nlp2(ex)

In [92]:
for name in nlpo.ents:
    if name.label_ == 'CHEMICAL':
        alldrugs.add(str(name))
nlpo = nlp(ex)
for name in nlpo.ents:
    if name.label_ == 'CHEMICAL':
        alldrugs.add(str(name))

alldrugs.add('Crocin')
alldrugs

{'Crocin', 'Nexplanon'}

In [117]:
'''Condition Finder If drug drug is recognized by nlp2'''
conditions = set()
rows = df_uci[df_uci['drugName'] == "Ortho Evra"]
for r in rows.iterrows():
    conditions.add(str(r[1][2]))
conditions

{'Birth Control'}